# Objective:

Create Dengue database metadata

# Built environment

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, StringType
from pyspark.sql.functions import year, month, col, sum, udf, substring, split, regexp_replace, when
import glob as gb
#from tqdm import tqdm
import re 
from functools import reduce
import numpy as np
import sys
import glob

In [2]:
sc = SparkContext()
sc.stop()

conf = SparkConf().setAppName("SINAM_dengue_cases")

conf = (conf.setMaster("local[*]")
       .set("spark.executor.memory", "3GB")
       .set("spark.driver.memory", "20GB"))

sc = SparkContext(conf = conf)
spark = SparkSession(sc)
spark

In [3]:
#define paths to files
spark = SparkSession.builder.appName("Dengue").getOrCreate()

# Save columns names of each dengue dataset

In [4]:
path = '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane'
filenames = glob.glob(path + "/*.csv")
filenames

['/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR21.csv',
 '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR09.csv',
 '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR08.csv',
 '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR20.csv',
 '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR18.csv',
 '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR19.csv',
 '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR00.csv',
 '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR14.csv',
 '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR15.csv',
 '/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGB

In [5]:
lst_df_columns = []
for f in filenames:
    print(f)
    #df = pd.read_csv(f, low_memory = False, encoding="iso-8859-1")  
    df = spark.read.format("csv").option("header",True).load(f)
    lst_df_columns.append([f[-12:-4]] + df.columns)

/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR21.csv
/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR09.csv
/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR08.csv
/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR20.csv
/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR18.csv
/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR19.csv
/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR00.csv
/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR14.csv
/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR15.csv
/Users/julianeoliveira/Documents/Projects/AESOP/AESOP datalake/Sinan_dengue/juliane/DENGBR01.csv
/Users/julianeoliveira/Documen

In [6]:
lst_df_columns

[['DENGBR21',
  'TP_NOT',
  'ID_AGRAVO',
  'DT_NOTIFIC',
  'SEM_NOT',
  'NU_ANO',
  'SG_UF_NOT',
  'ID_MUNICIP',
  'ID_REGIONA',
  'ID_UNIDADE',
  'DT_SIN_PRI',
  'SEM_PRI',
  'DT_NASC',
  'NU_IDADE_N',
  'CS_SEXO',
  'CS_GESTANT',
  'CS_RACA',
  'CS_ESCOL_N',
  'SG_UF',
  'ID_MN_RESI',
  'ID_RG_RESI',
  'ID_PAIS',
  'DT_INVEST',
  'ID_OCUPA_N',
  'FEBRE',
  'MIALGIA',
  'CEFALEIA',
  'EXANTEMA',
  'VOMITO',
  'NAUSEA',
  'DOR_COSTAS',
  'CONJUNTVIT',
  'ARTRITE',
  'ARTRALGIA',
  'PETEQUIA_N',
  'LEUCOPENIA',
  'LACO',
  'DOR_RETRO',
  'DIABETES',
  'HEMATOLOG',
  'HEPATOPAT',
  'RENAL',
  'HIPERTENSA',
  'ACIDO_PEPT',
  'AUTO_IMUNE',
  'DT_CHIK_S1',
  'DT_CHIK_S2',
  'DT_PRNT',
  'RES_CHIKS1',
  'RES_CHIKS2',
  'RESUL_PRNT',
  'DT_SORO',
  'RESUL_SORO',
  'DT_NS1',
  'RESUL_NS1',
  'DT_VIRAL',
  'RESUL_VI_N',
  'DT_PCR',
  'RESUL_PCR_',
  'SOROTIPO',
  'HISTOPA_N',
  'IMUNOH_N',
  'HOSPITALIZ',
  'DT_INTERNA',
  'UF',
  'MUNICIPIO',
  'TPAUTOCTO',
  'COUFINF',
  'COPAISINF',
  'COMUN

In [7]:
#search variable function

def check_variables(variable): 

    for value in lst_df_columns:
        print(value[0], variable in value)

In [59]:
check_variables('RESUL_VIRA')

DENGBR21 False
DENGBR09 False
DENGBR08 False
DENGBR20 False
DENGBR18 False
DENGBR19 False
DENGBR00 True
DENGBR14 False
DENGBR15 False
DENGBR01 True
DENGBR17 False
DENGBR03 True
DENGBR02 True
DENGBR16 False
DENGBR12 False
DENGBR06 True
DENGBR07 False
DENGBR13 False
DENGBR05 True
DENGBR11 False
DENGBR10 False
DENGBR04 True


In [111]:
lst_df_columns[6];

In [ ]:
['DENGBR00',
 'ID_DG_NOT',
 'ID_EV_NOT',
 'ANT_DT_INV',
 'OCUPACAO',
 'DENGUE',
 'ANO',
 'VACINADO',
 'DT_DOSE',
 'FEBRE',
 'DT_FEBRE',
 'DURACAO',
 'LACO',
 'CEFALEIA',
 'EXANTEMA',
 'DOR',
 'PROSTACAO',
 'MIALGIA',
 'NAUSEAS',
 'ARTRALGIA',
 'DIARREIA',
 'OUTROS',
 'SIN_OUT',
 'EPISTAXE',
 'PETEQUIAS',
 'GENGIVO',
 'METRO',
 'HEMATURA',
 'SANGRAM',
 'OUTROS_M',
 'OUTROS_M_D',
 'ASCITE',
 'PLEURAL',
 'PERICARDI',
 'ABDOMINAL',
 'HEPATO',
 'MIOCARDI',
 'HIPOTENSAO',
 'CHOQUE',
 'MANIFESTA',
 'INSUFICIEN',
 'OUTRO_S',
 'OUTRO_S_D',
 'ASCITE',
 'PLEURAL',
 'PERICARDI',
 'ABDOMINAL',
 'HEPATO',
 'MIOCARDI',
 'HIPOTENSAO',
 'CHOQUE',
 'MANIFESTA',
 'INSUFICIEN',
 'OUTRO_S',
 'OUTRO_S_D',
 'DT_CHOQUE',
 'HOSPITALIZ',
 'UF',
 'MUNICIPIO',
 'DT_COL_HEM',
 'HEMA_MAIOR',
 'DT_COL_PLQ',
 'PALQ_MAIOR',
 'DT_COL_HE2',
 'HEMA_MENOR',
 'DT_COL_PL2',
 'PLAQ_MENOR',
 'AMOS_OUT',
 'TECNICA',
 'RESUL_OUT',
 'CON_INF_MU',
 'CON_INF_UF',
 'CON_INF_PA',
 'CON_DOENCA',
 'CON_EVOLUC',
 'CON_DT_OBI',
 'CON_DT_ENC',
 'IN_AIDS']

# Check columns values

In [23]:
df = spark.read.format("csv").option("header",True).load(filenames[6]) #6

In [44]:
df.select('DT_SOROR1').show(20)

+-----------+
|  DT_SOROR1|
+-----------+
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
|b'        '|
+-----------+
only showing top 20 rows



In [45]:
df.select('DT_SOROR1').distinct().collect()

[Row(DT_SOROR1="b'20000615'"),
 Row(DT_SOROR1="b'20000327'"),
 Row(DT_SOROR1="b'20001118'"),
 Row(DT_SOROR1="b'20000427'"),
 Row(DT_SOROR1="b'20000727'"),
 Row(DT_SOROR1="b'20000308'"),
 Row(DT_SOROR1="b'20000310'"),
 Row(DT_SOROR1="b'20000224'"),
 Row(DT_SOROR1="b'20000717'"),
 Row(DT_SOROR1="b'20000323'"),
 Row(DT_SOROR1="b'20000514'"),
 Row(DT_SOROR1="b'20000131'"),
 Row(DT_SOROR1="b'20001231'"),
 Row(DT_SOROR1="b'20000802'"),
 Row(DT_SOROR1="b'20000130'"),
 Row(DT_SOROR1="b'20010215'"),
 Row(DT_SOROR1="b'20000406'"),
 Row(DT_SOROR1="b'20000210'"),
 Row(DT_SOROR1="b'20000720'"),
 Row(DT_SOROR1="b'20010913'"),
 Row(DT_SOROR1="b'20000609'"),
 Row(DT_SOROR1="b'20000614'"),
 Row(DT_SOROR1="b'20000901'"),
 Row(DT_SOROR1="b'20000529'"),
 Row(DT_SOROR1="b'20010108'"),
 Row(DT_SOROR1="b'20001220'"),
 Row(DT_SOROR1="b'20000519'"),
 Row(DT_SOROR1="b'20000906'"),
 Row(DT_SOROR1="b'        '"),
 Row(DT_SOROR1="b'20040519'"),
 Row(DT_SOROR1="b'20001205'"),
 Row(DT_SOROR1="b'20000202'"),
 Row(DT_